# Sesión 6 - Entrenar Word2vec y Doc2Vec desde 0

En este notebook vamos a ver cómo se entrena un modelo sencillo de word2vec eligiendo las dimensiones de los vectores.

Además, se creará un modelo de Doc2Vec a partir de un conjunto de documentos y se verá cómo se puede obtener las similitudes entre documentos.


In [1]:
# Instalamos gensim si no lo tenemos instalado
!pip3 install -U gensim
# Esto es por si no está ya instalado
!pip3 install -U pandas
!pip3 install -U nltk

# Apartado 1.1 Descargamos un corpus de prueba

Vamos a probar con un corpus de noticias que se encuentra en la URL https://valencia.inf.um.es/valencia-tgine/corpusNoticias.zip

In [2]:
# Descargamos un corpus de noticias que he creado
!wget --no-check-certificate https://valencia.inf.um.es/valencia-plne/corpusNoticias.zip
!unzip corpusNoticias.zip > extract.log

--2025-03-20 21:21:44--  https://valencia.inf.um.es/valencia-plne/corpusNoticias.zip
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4170052 (4.0M) [application/zip]
Saving to: ‘corpusNoticias.zip.2’

corpusNoticias.zip. 100%[===================>]   3.98M  4.38MB/s    in 0.9s    

2025-03-20 21:21:45 (4.38 MB/s) - ‘corpusNoticias.zip.2’ saved [4170052/4170052]

replace corpusNoticias/00005112614397266846.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Leemos todos lo ficheros y los metemos en una variable *texts*

Tened en cuenta que la codificación de caracteres en estos ficheros es UTF-8. Esto depende del contenido de las web a descargar.

In [3]:
from os import listdir
from os.path import isfile, join

my_path = "corpusNoticias/"
texts = []
for fn in listdir(my_path):
  f = open(my_path+fn, encoding = "utf-8")
  file_content = f.read()
  texts.append(file_content)
  f.close()

# Comprobar que se ha leído bien:
for text in texts[:3]:
  print(text)
  print("----------"*10)

El cambio de ley que permitirá trabajar a los estudiantes extranjeros: "Yo sobrevivía con 400 euros
Laura Rosero, inmigrante colombiana de 25 años, se trasladó a vivir a Madrid hace tres años para realizar un Máster en Cine. La joven había estudiado Comunicación Social y Periodismo en su país y, dado que su intención era trabajar en la producción de películas y cortos, pensó que mudarse a España sería un atajo para acceder al mercado laboral y obtener una mejor «calidad de vida». Nada más lejos de la realidad, porque cuando se instaló en la capital, aunque estaba cómoda y veía cómo gracias al esfuerzo académico había logrado mudarse a Europa, no tardó en darse cuenta de que no podía mantenerse por sí sola. «El visado de estudiante no me permitía trabajar con oportunidades más allá de un contrato en prácticas, así que sobrevivía con 400 euros de beca», explica. La joven describe ese año de máster como una época «difícil» y reconoce que, con ese presupuesto, «no le llegaba para vivir». «

# Apartado 1.2 Entrenamos un modelo word2vec a partir del corpus

Aquí vamos a entrenar un modelo word2vec con la librería GENSIM. Como Tokenizer se utilizará el word_tokenize de NLTK, pero se podría usar cualquier otro tokenizador.

In [4]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (792 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: joblib
    Found

In [5]:
from os import listdir
from os.path import isfile, join
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
from nltk.tokenize import word_tokenize


In [6]:
# Procesamos todos los textos y le aplicamos el word_tokenize de NLTK
nltk.download('punkt_tab')
train_texts=[]
for text in texts:
     train_texts.append(word_tokenize(text.lower()))

# Comprobar tokenización:
print()
print()
for t in train_texts[:3]:
  print(t)
  print("----------"*10)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.




['el', 'cambio', 'de', 'ley', 'que', 'permitirá', 'trabajar', 'a', 'los', 'estudiantes', 'extranjeros', ':', '``', 'yo', 'sobrevivía', 'con', '400', 'euros', 'laura', 'rosero', ',', 'inmigrante', 'colombiana', 'de', '25', 'años', ',', 'se', 'trasladó', 'a', 'vivir', 'a', 'madrid', 'hace', 'tres', 'años', 'para', 'realizar', 'un', 'máster', 'en', 'cine', '.', 'la', 'joven', 'había', 'estudiado', 'comunicación', 'social', 'y', 'periodismo', 'en', 'su', 'país', 'y', ',', 'dado', 'que', 'su', 'intención', 'era', 'trabajar', 'en', 'la', 'producción', 'de', 'películas', 'y', 'cortos', ',', 'pensó', 'que', 'mudarse', 'a', 'españa', 'sería', 'un', 'atajo', 'para', 'acceder', 'al', 'mercado', 'laboral', 'y', 'obtener', 'una', 'mejor', '«', 'calidad', 'de', 'vida', '»', '.', 'nada', 'más', 'lejos', 'de', 'la', 'realidad', ',', 'porque', 'cuando', 'se', 'instaló', 'en', 'la', 'capital', ',', 'aunque', 'estaba', 'cómoda', 'y', 'veía', 'cómo', 'gracias', 'al', 'esfuerzo', 'académico', 'había', 'l

In [7]:
from gensim.models import Word2Vec
# define training data
# train model
# se puede entrenar el modelo con distintos parámetros como el tamaño del vector,
# tamaño de la ventana, las veces que debe aparecer una palabra, etc.
model = Word2Vec(train_texts, vector_size=100, window=10, min_count=1, workers=10)
# summarize the loaded model
print(model)
# save model
model.save('model.bin')


Word2Vec<vocab=65286, vector_size=100, alpha=0.025>


In [8]:
# Cargamos el modelo guardado
new_model = Word2Vec.load('model.bin')

# Probamos el nuevo modelo
# Imprimimos el vector de la palabra 'energía'
print(model.wv['energía'])


[-0.54966843  0.0875638   0.49173936  0.31768817  0.60506946 -1.3478377
  0.44486743  1.3436048  -0.22349086 -1.2903141   0.18226402 -0.63643825
 -0.20991665 -0.10744024  0.21418516 -0.33001572  1.280824   -0.6162934
  0.27269593 -0.6326234   0.848064    0.95154965  0.7661901  -0.5198638
  0.11636665 -0.91778165 -0.6305624   0.5948861   0.05202897  0.23123802
  1.1733158  -0.72866476 -0.00409259  0.26516086  0.43973595  0.36105534
  0.1902662   0.08324517  0.844308   -0.03544473  0.5116959  -0.2893432
 -0.11126821  0.01203209 -0.03792691  1.171641   -0.21934138  0.2378036
  0.4355185   0.21893078 -0.29556614 -0.04398661  0.14149816 -0.9219015
  0.06030832  1.4807315   0.09648234  0.23586525  0.85574925  0.2006561
 -0.6391146  -0.34830517  0.26138386  0.10251787 -0.38559857  0.9993626
  0.7503709   0.09393875 -1.4534297   0.5084284  -0.2581475   0.11072753
  1.4035052   0.10173576 -0.26802996 -0.2895834   0.70822775 -0.33866778
 -0.9215679   0.92181545 -1.0497291  -0.4286957   0.9014103

In [9]:
# Probamos similitudes
new_model.wv.similarity("coronavirus", "covid")

0.9616275

In [10]:
# Probamos en listar alguna de las palabras más similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'energía'
palabra = 'energía'
print(new_model.wv.most_similar(palabra))

[('coronavirus', 0.9616276025772095), ('covid-19', 0.9595453143119812), ('inmunización', 0.897813618183136), ('contagio', 0.8871909976005554), ('vacunación', 0.8726752996444702), ('pfizer', 0.8726686239242554), ('astrazeneca', 0.8609094023704529), ('casos', 0.8549132347106934), ('sars-cov-2', 0.8539988398551941), ('pacientes', 0.851979672908783)]
[('producción', 0.9278727173805237), ('eficiencia', 0.9064045548439026), ('inversión', 0.9029995799064636), ('capacidad', 0.9011706709861755), ('electricidad', 0.9000114798545837), ('consecución', 0.8928479552268982), ('mejora', 0.8907243609428406), ('emisiones', 0.8906304836273193), ('carbono', 0.8856784701347351), ('reducir', 0.8841058015823364)]


In [11]:
# Probamos alguna analogía
# Covid es a Vacunas lo que Salud es a ...
print(new_model.wv.most_similar(positive=["salud", "vacunas"], negative=["covid"], topn=10))

# Covid es a Vacunas lo que Guerra es a ...
print(new_model.wv.most_similar(positive=["guerra", "vacunas"], negative=["covid"], topn=10))


[('pública', 0.8515499234199524), ('medidas', 0.8274768590927124), ('seguridad', 0.8245885968208313), ('cavaleri', 0.8233178853988647), ('administración', 0.8051121234893799), ('sanidad', 0.803527295589447), ('prevención', 0.7982622981071472), ('ema', 0.797874391078949), ('consejerías', 0.7913883328437805), ('educación', 0.7817356586456299)]
[('proactivos', 0.8448898196220398), ('política', 0.8444766998291016), ('importancia', 0.8441581130027771), ('cuestión', 0.839758574962616), ('voluntad', 0.8394683003425598), ('convivencia', 0.8384665250778198), ('democracia', 0.8341692090034485), ('opinión', 0.8326576948165894), ('estrategia', 0.8308067917823792), ('tomar', 0.8279587030410767)]


In [12]:
# Probamos a mostrar términos similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'ucrania'
palabra = 'ucrania'
print(new_model.wv.most_similar(palabra))

[('coronavirus', 0.9616276025772095), ('covid-19', 0.9595453143119812), ('inmunización', 0.897813618183136), ('contagio', 0.8871909976005554), ('vacunación', 0.8726752996444702), ('pfizer', 0.8726686239242554), ('astrazeneca', 0.8609094023704529), ('casos', 0.8549132347106934), ('sars-cov-2', 0.8539988398551941), ('pacientes', 0.851979672908783)]
[('particular', 0.8929041028022766), ('rusia', 0.8905982971191406), ('bruselas', 0.8880709409713745), ('ciudadanía', 0.8839694857597351), ('guerra', 0.8834167122840881), ('violencia', 0.8809518814086914), ('invasión', 0.8753531575202942), ('personal', 0.8731408715248108), ('conflictos', 0.8684223890304565), ('urgente', 0.8645603656768799)]


# Apartado 1.3 Entrenamos un Doc2Vec con los mismos textos

El Doc2Vec se puede ver como un tipo de sentence embeddings que traduce todo el texto a un vector de unas dimensiones determinadas.


In [13]:
#Import all the dependencies
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

#Necestiamos crear un TaggedDocument para cada uno de los textos indicando un índice de cada texto
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(train_texts)]

# Comprobar tokenización:
print()
print()
for td in tagged_data[:3]:
  print(td)
  print("----------"*10)



TaggedDocument<['el', 'cambio', 'de', 'ley', 'que', 'permitirá', 'trabajar', 'a', 'los', 'estudiantes', 'extranjeros', ':', '``', 'yo', 'sobrevivía', 'con', '400', 'euros', 'laura', 'rosero', ',', 'inmigrante', 'colombiana', 'de', '25', 'años', ',', 'se', 'trasladó', 'a', 'vivir', 'a', 'madrid', 'hace', 'tres', 'años', 'para', 'realizar', 'un', 'máster', 'en', 'cine', '.', 'la', 'joven', 'había', 'estudiado', 'comunicación', 'social', 'y', 'periodismo', 'en', 'su', 'país', 'y', ',', 'dado', 'que', 'su', 'intención', 'era', 'trabajar', 'en', 'la', 'producción', 'de', 'películas', 'y', 'cortos', ',', 'pensó', 'que', 'mudarse', 'a', 'españa', 'sería', 'un', 'atajo', 'para', 'acceder', 'al', 'mercado', 'laboral', 'y', 'obtener', 'una', 'mejor', '«', 'calidad', 'de', 'vida', '»', '.', 'nada', 'más', 'lejos', 'de', 'la', 'realidad', ',', 'porque', 'cuando', 'se', 'instaló', 'en', 'la', 'capital', ',', 'aunque', 'estaba', 'cómoda', 'y', 'veía', 'cómo', 'gracias', 'al', 'esfuerzo', 'académic

In [14]:
# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 5
vec_size = 100
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm = 1,
                epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    doc2vec_model.train(tagged_data,
                total_examples=doc2vec_model.corpus_count,
                epochs=doc2vec_model.epochs)
    # decrease the learning rate
    doc2vec_model.alpha -= 0.0002
    # fix the learning rate, no decay
    doc2vec_model.min_alpha = model.alpha

doc2vec_model.save("d2v.model")
print("Model Saved")

Model Saved


In [15]:
from gensim.models import Doc2Vec

doc2vec_model= Doc2Vec.load("d2v.model")
# Probamos a encontrar textos similares a uno dado
query_text = "nadal"
test_data = word_tokenize(query_text)
v1 = doc2vec_model.infer_vector(test_data)

# Encontramos los documentos más similares
similar_doc = doc2vec_model.dv.most_similar(v1)

# Imprimimos los 5 documentos más similares
top5_similar_doc = similar_doc[:5]
print(top5_similar_doc)
for doc in top5_similar_doc:
  print("--------------------------")
  print(texts[int(doc[0])])
  print('Similitud:',doc[1])



[('1058', 0.6124743819236755), ('1422', 0.6007329821586609), ('861', 0.5957950353622437), ('1509', 0.593778669834137), ('1054', 0.590646505355835)]
--------------------------
Rafa Nadal se retira de la Copa Laver por "motivos personales"
Rafa Nadal se ha retirado de la Copa Laver, en la que el viernes disputó junto a Roger Federer el partido de dobles, el último de la carrera del suizo, por "motivos personales", confirmó la organización del torneo. Según apuntan algunas fuentes, el campeón de 22 majors volará a Mallorca para acompañar a su mujer, con la que espera su primer hijo. En la rueda de prensa posterior a su derrota del viernes ante Jack Sock y Francis Tiafoe, Nadal admitió que no se encontraba en absoluto en su mejor momento. "No estoy bien. No voy a jugar. Tengo un conflicto interno bastante importante y ahora mismo no os puedo contestar, se me hace difícil. Cuando terminen todos estos momentos de emoción volveré a mi habitación y veré qué es lo que realmente tengo que hacer"

Podemos ahora sacar el vector de una frase de ejemplo con el método ```infer_vector``` del modelo Doc2Vec

In [16]:
frase = "Las vacunas son una muy buena solución para el COVID"
vector = doc2vec_model.infer_vector(word_tokenize(frase))
print(vector)



[-0.09300517  0.13779175 -0.08150215  0.11505105 -0.11832687 -0.03355703
 -0.03556609  0.21398129 -0.17638782  0.07347623 -0.06622016 -0.03756933
 -0.11138067  0.12688838  0.13176124 -0.04352955  0.11552393  0.02075611
 -0.10841041 -0.09238128  0.05245872  0.04062342  0.12578683  0.06922287
 -0.03091728  0.11775588 -0.15422659 -0.08668992 -0.1423299  -0.16109878
  0.14400291  0.05284026  0.15638171  0.02689828  0.02614942 -0.05249069
  0.00832978 -0.03303083 -0.01849047 -0.09177777  0.16557693  0.10032246
 -0.09673145 -0.21017034  0.05269767 -0.0019765  -0.09246822  0.00804257
  0.11234571  0.01009793 -0.05100893 -0.07525957 -0.09857091 -0.10605402
 -0.10660521  0.01688475 -0.00601662  0.02395856 -0.23491445  0.01607885
  0.19299448  0.15672156 -0.03395867 -0.07131705 -0.00132948  0.1830033
 -0.01740997  0.09517356 -0.09897375  0.01730702 -0.03537436 -0.00852644
  0.09021304 -0.00680741  0.03315742  0.03579649  0.02518085 -0.03657433
  0.00530994 -0.04579997 -0.02677135  0.00473733 -0.

## Ejercicio a resolver

### 1.- Con el nuevo Doc2Vec entrenado para generar sentence embeddings probar cómo funciona la clasificación de los sentence embeddings del conjunto de entrenamiento `dataset_train.csv`y de `dataset_test`.

(Si no sabe cómo empezar, veáse apartado 1.8 de práctica 6_1_Word_embeddings ...)

In [17]:
# Descargamos los datasets en español que hemos usado en otras prácticas
!wget -c --no-check-certificate https://valencia.inf.um.es/valencia-plne/dataset_train.csv
!wget -c --no-check-certificate https://valencia.inf.um.es/valencia-plne/dataset_test.csv


--2025-03-20 21:25:06--  https://valencia.inf.um.es/valencia-plne/dataset_train.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1367959 (1.3M) [text/csv]
Saving to: ‘dataset_train.csv’

dataset_train.csv   100%[===================>]   1.30M  1.88MB/s    in 0.7s    

2025-03-20 21:25:07 (1.88 MB/s) - ‘dataset_train.csv’ saved [1367959/1367959]

--2025-03-20 21:25:07--  https://valencia.inf.um.es/valencia-plne/dataset_test.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584195 (571K) [text/csv]
Saving to: ‘dataset_test.csv’

dataset_test.csv    100%[===================>] 570.50K   875KB/s    in 0.7s    

2025-03-20 21:25:08 (875 KB/s) - ‘dataset_test.csv’ save

In [21]:
import pandas
import numpy as np

df_train = pandas.read_csv("dataset_train.csv",encoding="UTF-8")
df_test = pandas.read_csv("dataset_test.csv",encoding="UTF-8")

# Ponemos en lower_case los dos conjuntos de tweets
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

df_train['tweet'] = df_train['tweet'].str.lower()
df_test['tweet'] = df_test['tweet'].str.lower()

# Generamos los embeddings para los tweets de entrenamiento y prueba
train_embeddings = [doc2vec_model.infer_vector(word_tokenize(tweet)) for tweet in df_train['tweet']]
test_embeddings = [doc2vec_model.infer_vector(word_tokenize(tweet)) for tweet in df_test['tweet']]

# Entrenams un clasificador SVM con los embeddings generados
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(train_embeddings, df_train['label'])

# Realizamos predicciones en el conjunto de prueba
predict = clf.predict(test_embeddings)

# Calculamos y mostramos el rendimiento del modelo
accuracy = np.mean(predict == df_test['label'])
print(f"Accuracy = {accuracy:.4f}")
print("\nReporte de clasificación:\n")
print(classification_report(df_test['label'], predict))

Accuracy = 0.7438

Reporte de clasificación:

              precision    recall  f1-score   support

    negative       0.67      0.37      0.47       561
    positive       0.76      0.92      0.83      1227

    accuracy                           0.74      1788
   macro avg       0.71      0.64      0.65      1788
weighted avg       0.73      0.74      0.72      1788



### 2.- Entrena ahora otro Doc2Vec ampliando las dimensiones y compara los resultados.

In [22]:
# Primero preparamos los datos para el nuevo entrenamiento
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str]) for i, doc in enumerate(texts)]

#Entrenamos el nuevo Doc2Vec con 300 dimensiones

# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 10
vec_size = 200
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                        alpha=alpha,
                        min_alpha=0.00025,
                        min_count=1,
                        dm = 1,
                        epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print(f"Época {epoch+1}")
    doc2vec_model.train(tagged_data,
                        total_examples=doc2vec_model.corpus_count,
                        epochs=doc2vec_model.epochs)
    # Reducimos la tasa de aprendizaje
    doc2vec_model.alpha -= 0.0002
    # Fijamos la tasa de aprendizaje, sin decaimiento
    doc2vec_model.min_alpha = doc2vec_model.alpha

# Guardamos el nuevo modelo entrenado
doc2vec_model.save("d2v_300d.model")
print("Hecho ✅")

# Generamos nuevos embeddins para los conjuntos de entrenamiento y prueba
train_embeddings = [doc2vec_model.infer_vector(word_tokenize(tweet)) for tweet in df_train['tweet']]
test_embeddings = [doc2vec_model.infer_vector(word_tokenize(tweet)) for tweet in df_test['tweet']]

# Entrenamos un clasificador SVM con los nuevos embeddins
clf_300d = LinearSVC(random_state=0, tol=1e-5)
clf_300d.fit(train_embeddings, df_train['label'])

# Realizamos predicciones en el conjunto de prueba
predict_300d = clf_300d.predict(test_embeddings)

# Calculamos y mostramos el rendimiento del modelo
accuracy = np.mean(predict_300d == df_test['label'])
print(f"Accuracy = {accuracy:.4f}")
print("\nReporte de clasificación:\n")
print(classification_report(df_test['label'], predict_300d))

Época 1


Época 2
Época 3
Época 4
Época 5
Época 6
Época 7
Época 8
Época 9
Época 10
Hecho ✅
Accuracy = 0.7416

Reporte de clasificación:

              precision    recall  f1-score   support

    negative       0.64      0.40      0.49       561
    positive       0.77      0.90      0.83      1227

    accuracy                           0.74      1788
   macro avg       0.70      0.65      0.66      1788
weighted avg       0.73      0.74      0.72      1788



Ha habido una leve empeoramiento en la precisión del modelo después de ampliar las dimensiones:

- Primero modelo: `accuracy=0.7438`
- Nuevo modelo: `accuracy=0.7416`